# 🔧 Checkpoint 2: Criando Tools para Agentes

## Objetivo
O foco aqui é dar "superpoderes" aos agentes. Sozinhos, eles só podem conversar. Com ferramentas (Tools), eles podem executar ações no mundo real, como pedir input do usuário ou chamar outros agentes.


## 📦 Configuração de ambiente

> *Vamos carregar tudo que fizemos no Checkpoint 1!*


In [ ]:
# Instalar as bibliotecas
!pip install agno openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.2 MB/s eta 0:00:00


In [ ]:

import os
from typing import Dict
import json
from google.colab import userdata


os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

print("✅ Ambiente configurado!")


✅ Ambiente configurado!


## 📚 Definindo o material
> Este texto é o "cérebro" ou a "fonte da verdade" para os nossos agentes. O Coordenador usará este material para criar perguntas, e o Avaliador o usará como gabarito para julgar as respostas do aluno. Isso garante que a IA se atenha ao escopo do que queremos ensinar.

In [ ]:
material = """
# Git - Controle de Versão

Git é um sistema de controle de versão que registra mudanças no código ao longo do tempo.

## Conceitos Principais

**Repositório (repo):** Pasta que contém seu projeto e todo histórico de mudanças.

**Commit:** Registro de uma mudança no código. Como um "save point" do seu trabalho.
Cada commit tem mensagem explicando o que mudou.

**Branch (ramificação):** Linha independente de desenvolvimento.
- main/master: branch principal (código estável)
- feature branches: para desenvolver novas funcionalidades

**Comandos básicos:**
- git init: cria novo repositório
- git add: marca arquivos para próximo commit
- git commit -m "mensagem": salva mudanças com descrição
- git status: mostra estado atual (o que mudou)
- git log: histórico de commits
- git branch: lista/cria branches
- git checkout: muda de branch
- git merge: junta mudanças de uma branch em outra

**Workflow típico:**
1. Modificar arquivos
2. git add (preparar mudanças)
3. git commit (salvar mudanças)
4. git push (enviar para servidor remoto)

**Por que usar Git?**
- Histórico completo de mudanças
- Trabalho em equipe sem conflitos
- Experimente código sem medo (sempre pode voltar)
- Backup automático do trabalho
"""

## 🤖 Criação dos agentes

> Agora vem a parte interessante! Vamos criar nossos dois agentes

- name: Dá um nome ao agente, útil para identificação.
- model=Gemini(id="gemini-2.5-flash"): Especifica o modelo a ser usado. O "flash" é uma versão rápida e eficiente.
- instructions=[...]: Esta é a parte mais crítica. É o prompt do sistema, ou a "personalidade" do agente.

In [ ]:
from agno.agent import Agent
from agno.models.google import Gemini
from agno.tools import tool

# Funções dos agentes (versão simplificada para demo)
def criar_agente_coordenador(material: str) -> Agent:
    return Agent(
        name="Coordenador Pedagógico",
        model=Gemini(id="gemini-2.5-flash"),
        instructions=[
            f"MATERIAL: {material}",
            "Você é um tutor conversacional.",
            "Faça perguntas sobre o material.",
            "Seja encorajador e amigável."
        ]
    )

def criar_agente_avaliador(material: str) -> Agent:
    return Agent(
        name="Avaliador Rigoroso",
        model=Gemini(id="gemini-2.5-flash"),
        instructions=[
            f"MATERIAL: {material}",
            "Avalie respostas: BOM, PARCIAL ou RUIM",
            "Seja justo mas rigoroso."
        ]
    )

# Criar agentes
coordenador = criar_agente_coordenador(material)
avaliador = criar_agente_avaliador(material)

print("✅ Agentes carregados!")
print(f"🤖 Coordenador: {coordenador.name}")
print(f"⚖️ Avaliador: {avaliador.name}")


✅ Agentes carregados!
🤖 Coordenador: Coordenador Pedagógico
⚖️ Avaliador: Avaliador Rigoroso


## Estado compartilhado
> *Agora vamos configurar o estado que nossos agentes vão compartilhar!*

Este estado vai servir para nos ajudar a monitorar o progresso do aluno a cada tópico e controlar o fluxo da avaliação pra que ela tenha um fim

In [ ]:
# Estado compartilhado entre agentes
_estado = {
    "agente_avaliador": avaliador,
    "historico": [],
    "tracking": {},
    "num_perguntas": 0,
    "max_perguntas": 6
}


## 🧰 Tool (ferramenta)

@tool:  Este é um "decorador". Quando ativado, ele sinaliza para a biblioteca agno que esta função não é uma função comum, mas sim uma ferramenta que o agente de IA pode escolher executar.

In [ ]:

#@tool
def fazer_pergunta(pergunta: str) -> Dict:
    """
    Apresenta pergunta ao aluno e obtém resposta via input humano.

    Args:
        pergunta: Pergunta a ser feita ao aluno

    Returns:
        dict com 'resposta' e 'numero_pergunta' e 'perguntas_restantes'
    """

    _estado["num_perguntas"] += 1
    num = _estado["num_perguntas"]

    print(f"\n🤖 TUTOR (Pergunta {num}): {pergunta}")
    resposta = input("👤 VOCÊ: ")

    # Registrar pergunta
    _estado["historico"].append({
        "tipo": "pergunta",
        "numero": num,
        "conteudo": pergunta
    })

    # Registrar resposta
    _estado["historico"].append({
        "tipo": "resposta",
        "numero": num,
        "conteudo": resposta
    })

    # Avisar quando estiver próximo do limite
    restantes = _estado["max_perguntas"] - num
    aviso = ""
    if restantes <= 2:
        aviso = f" [Restam apenas {restantes} perguntas!]"

    return {
        "resposta": resposta,
        "numero_pergunta": num,
        "perguntas_restantes": restantes
    }

### 🛠️ Outras tools importantes


In [ ]:
def exibir_feedback(feedback: str) -> str:
    """
    Exibe feedback BREVE ao aluno na tela.

    Use esta tool para mostrar seu feedback após avaliar a resposta do aluno.

    Args:
        feedback: Mensagem BREVE de feedback para o aluno

    Returns:
        Confirmação de que o feedback foi exibido
    """
    print(f"\n 💬 TUTOR: {feedback}\n")

    # Registrar feedback no histórico
    _estado["historico"].append({
        "tipo": "feedback",
        "conteudo": feedback
    })

    return "Feedback exibido ao aluno"


def solicitar_avaliacao(pergunta: str, resposta: str, topico: str) -> Dict:
    """
    Solicita avaliação da resposta ao Agente Avaliador.

    Args:
        pergunta: Pergunta feita
        resposta: Resposta do aluno
        topico: Tópico sendo avaliado

    Returns:
        dict com avaliação: {'nivel', 'justificativa', 'proximo_passo'}
    """
    avaliador = _estado["agente_avaliador"]

    if avaliador is None:
        return {
            "nivel": "sabe_parcial",
            "justificativa": "Avaliador não disponível",
            "proximo_passo": "avançar"
        }

    # Contar tentativas no tópico
    tentativas = _estado["tracking"].get(topico, {}).get("tentativas", 0)

    # Solicitar avaliação ao agente
    response = avaliador.run(
        f"TÓPICO: {topico}\n"
        f"TENTATIVAS NO TÓPICO: {tentativas + 1}\n\n"
        f"PERGUNTA: {pergunta}\n\n"
        f"RESPOSTA DO ALUNO: {resposta}\n\n"
        f"Avalie esta resposta comparando com o material de referência.\n"
        f"Responda APENAS com JSON no formato especificado."
    )

    # Extrair JSON
    try:
        conteudo = response.content.strip()
        conteudo = conteudo.replace('```json', '').replace('```', '').strip()
        avaliacao = json.loads(conteudo)

        # Atualizar tracking
        if topico not in _estado["tracking"]:
            _estado["tracking"][topico] = {"tentativas": 0, "status": "em_avaliacao"}

        _estado["tracking"][topico]["tentativas"] += 1
        _estado["tracking"][topico]["ultima_avaliacao"] = avaliacao

        return avaliacao

    except json.JSONDecodeError:
        # Fallback
        return {
            "nivel": "sabe_parcial",
            "justificativa": "Erro ao processar avaliação",
            "proximo_passo": "avançar"
        }


@tool
def registrar_decisao(decisao: str, topico: str, motivo: str) -> str:
    """
    Registra decisão do Coordenador sobre próximo passo.

    Args:
        decisao: 'avançar', 'aprofundar', ou 'encerrar'
        topico: Tópico atual
        motivo: Justificativa da decisão

    Returns:
        Confirmação do registro
    """
    if topico in _estado["tracking"]:
        _estado["tracking"][topico]["decisao"] = decisao
        _estado["tracking"][topico]["motivo"] = motivo

        # Se avançar, marcar status final
        if decisao == "avançar":
            ultima_aval = _estado["tracking"][topico].get("ultima_avaliacao", {})
            nivel = ultima_aval.get("nivel", "sabe_parcial")

            if nivel == "nao_sabe":
                _estado["tracking"][topico]["status"] = "lacuna"
            elif nivel == "sabe_parcial":
                _estado["tracking"][topico]["status"] = "parcial"
            else:
                _estado["tracking"][topico]["status"] = "dominado"

    return f"✓ Decisão registrada: {decisao} ({motivo})"


@tool
def obter_contexto_recente() -> Dict:
    """
    Obtém contexto recente para formular pergunta contextual.
    """
    # Últimas 5 interações apenas
    ultimos_items = _estado["historico"][-5:] if len(_estado["historico"]) > 5 else _estado["historico"]

    contexto = []
    for item in ultimos_items:
        if item["tipo"] == "pergunta":
            contexto.append(f"PERGUNTA: {item['conteudo']}")
        elif item["tipo"] == "resposta":
            contexto.append(f"RESPOSTA: {item['conteudo']}")
        elif item["tipo"] == "feedback":
            contexto.append(f"FEEDBACK: {item['conteudo']}")

    return {
        "ultimas_interacoes": "\n".join(contexto) if contexto else "Nenhuma interação ainda",
        "perguntas_restantes": _estado["max_perguntas"] - _estado["num_perguntas"],
        "instrucao": "Use este contexto para formular pergunta adaptativa baseada nas últimas interações."
    }


@tool
def obter_historico_completo() -> Dict:
    """
    Obtém TODO o histórico de perguntas, respostas e feedback da avaliação.

    Use esta tool ANTES de gerar o relatório final para ter acesso a:
    - Todas as perguntas feitas
    - Todas as respostas do aluno
    - Todo o feedback dado
    - Status dos tópicos avaliados

    Isso permite gerar relatório PERSONALIZADO com exemplos reais das respostas.

    Returns:
        dict com 'historico_formatado', 'tracking' e 'estatisticas'
    """
    # Formatar histórico de forma legível
    historico_formatado = []

    for item in _estado["historico"]:
        if item["tipo"] == "pergunta":
            historico_formatado.append(
                f"PERGUNTA {item.get('numero', '?')}: {item['conteudo']}"
            )
        elif item["tipo"] == "resposta":
            historico_formatado.append(
                f"RESPOSTA DO ALUNO: {item['conteudo']}"
            )
        elif item["tipo"] == "feedback":
            historico_formatado.append(
                f"FEEDBACK DADO: {item['conteudo']}"
            )

    # Formatar tracking de tópicos
    tracking_formatado = {}
    for topico, info in _estado["tracking"].items():
        status = info.get("status", "nao_avaliado")
        tentativas = info.get("tentativas", 0)
        tracking_formatado[topico] = {
            "status": status,
            "tentativas": tentativas
        }

    # Estatísticas
    estatisticas = {
        "total_perguntas": _estado["num_perguntas"],
        "max_perguntas": _estado["max_perguntas"],
        "topicos_avaliados": len(_estado["tracking"])
    }

    return {
        "historico_formatado": "\n\n".join(historico_formatado),
        "tracking": tracking_formatado,
        "estatisticas": estatisticas,
        "instrucao": "Use este histórico completo para gerar relatório PERSONALIZADO baseado nas respostas reais do aluno. NÃO é necessário citar as respostas literalmente, apenas use-as para construir um relatório contextualizado."
    }


@tool
def salvar_relatorio(conteudo: str) -> str:
    """
    Salva relatório em arquivo markdown com timestamp automático.

    Args:
        conteudo: Conteúdo do relatório em markdown
    """
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    nome_arquivo = f"relatorio_aprendizagem_{timestamp}.md"

    try:
        # Salvar arquivo
        with open(nome_arquivo, 'w', encoding='utf-8') as f:
            f.write(conteudo)

        # EXIBIR relatório na tela
        print("\n" + "="*70)
        print("RELATÓRIO FINAL DE APRENDIZAGEM")
        print("="*70 + "\n")
        print(conteudo)
        print("\n" + "="*70)
        print(f"Relatório salvo em: {nome_arquivo}")
        print("="*70 + "\n")

        return f"Relatório exibido e salvo: {nome_arquivo}"
    except Exception as e:
        return f"Erro ao salvar: {str(e)}"


def configurar_estado(agente_avaliador, max_perguntas=10):
    """Configura estado compartilhado (chamado pelo orchestrator)"""
    _estado["agente_avaliador"] = agente_avaliador
    _estado["max_perguntas"] = max_perguntas

## 🧪 Testar tool

> *Agora vamos simular um ciclo completo de interação para validar as ferramentas*


In [ ]:

print("🧪 Testando nossa tool...")


# Testar fazer_pergunta
resultado = fazer_pergunta("O que é Git?")
print(f"\n📝 Resultado da tool: {resultado}")

# Testar solicitar_avaliacao
avaliacao = solicitar_avaliacao("O que é Git?", "Git é controle de versão", "Git Básico")
print(f"\n📊 Avaliação: {avaliacao}")

# Testar exibir_feedback
feedback_result = exibir_feedback("Correto! Você captou a essência do Git.")
print(f"\n💬 Feedback: {feedback_result}")



🧪 Testando nossa tool...

🤖 TUTOR (Pergunta 1): O que é Git?
👤 VOCÊ: Git é controle de versão

📝 Resultado da tool: {'resposta': 'Git é controle de versão', 'numero_pergunta': 1, 'perguntas_restantes': 5}

📊 Avaliação: {'avaliacao': 'PARCIAL', 'comentario': "A resposta está correta ao identificar Git como um 'controle de versão', o que é a essência do Git. No entanto, o material de referência complementa essa definição explicando que ele 'registra mudanças no código ao longo do tempo', o que adicionaria mais contexto e completude à resposta."}

 💬 TUTOR: Correto! Você captou a essência do Git.


💬 Feedback: Feedback exibido ao aluno


## 🎉 Resumo do Checkpoint 2

### ✅ O que aprendemos:
- **Tools** com `@tool`
- **Estado compartilhado** entre agentes

### 🎯 Próximo passo:
**Checkpoint 3:** Vamos fazer os agentes trabalharem juntos!

